In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import array_to_img
from PIL import Image
from tensorflow.keras.preprocessing import image
import re
import os
import random
from pathlib import Path
from google.colab import drive
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D
import cv2
from tensorflow.keras import regularizers
from sklearn.decomposition import PCA
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D

In [ ]:
#outputs
def y(path):
  df=pd.read_csv(path)
  y=df.iloc[:,-1]
  return np.array(y)
y_train=y('/content/pendigits_train.csv')
y_test=y('/content/pendigits_test.csv')

In [ ]:
#images from coordinates
from dataclasses import dataclass
from typing import Optional
@dataclass
class CoordToImg:

  X: str #filepath of x-train/test
  y_train: Optional[np.array]
  y_test: Optional[np.array]
  train: Optional[bool]=False
  test: Optional[bool]=False

  #img to arr
  def __post_init__(self):
    self.execute()
  
  def workDir(self):
    if self.train:
      os.chdir('/content/drive/MyDrive/digit_train')
    elif self.test:
      os.chdir('/content/drive/MyDrive/digit_test')

  #creating a white img
  def white_img(self):
    img=np.zeros((500,500,3)) #size : tuple=(500,500,3)
    img.fill(255)
    return img
  
  def coordinatesList(self,X):
    with open(X) as txt:
      l=[]
      for line in txt.readlines():
        x=re.search('[a-zA-Z]',line)
        if(x==None):
          l.append(re.findall('[0-9]+',line))

    for x in l:
      for i in range(2):
        if x:
          x[i]=int(x[i])
        else:
          continue
    l.append([])
    return l

  def saveImage(self):
    image=self.white_img()
    coordList=self.coordinatesList(self.X)
    i=0
    for x in coordList:
      if x:
        image[x[0],x[1]]=(0,0,0)
      else:
        im=array_to_img(image)
        im=im.rotate(-270)
        if self.train:
          im.save('/content/drive/MyDrive/digit_train/{}/{}.jpg'.format(self.y_train[i],i))
        else:
          im.save('/content/drive/MyDrive/digit_test/{}/{}.jpg'.format(self.y_test[i],i))
        i+=1
        image=self.white_img()
        continue

  def execute(self):
    l=self.coordinatesList(self.X)
    self.workDir()
    self.saveImage()

In [ ]:
train_Data=CoordToImg('/content/pendigits-orig.tra',y_train,y_test,train=True) #7494 records

In [ ]:
test_Data=CoordToImg('/content/pendigits-orig.tes',y_train,y_test,test=True) #3498 records

In [ ]:
#Data Preprocessing and Model
from typing import Optional
from dataclasses import dataclass
@dataclass
class ConvoNeuralNets:

  PCA: Optional[bool]=False
  
  def __post_init__(self):
    self.execute()
  
  def workDir(self):
    os.chdir('/content/drive/MyDrive/digit_train')
    return os.getcwd()

  def preprocessData(self): #converts image into array
    self.trainData=[]
    self.testData=[]
    categories=[0,1,2,3,4,5,6,7,8,9]

    for category in categories:
      path=os.path.join('/content/drive/MyDrive/digit_train',str(category))
      for filename in os.listdir(path):
        image_array=cv2.imread(os.path.join(path,filename),cv2.IMREAD_GRAYSCALE)
        if self.PCA:
          image_array=cv2.resize(image_array,(150,150))
        if not self.PCA:
          image_array=cv2.resize(image_array,(128,128))
        self.trainData.append([image_array,category])

    for category in categories:
      path=os.path.join('/content/drive/MyDrive/digit_test',str(category))
      for filename in os.listdir(path):
        image_testarr=cv2.imread(os.path.join(path,filename),cv2.IMREAD_GRAYSCALE)
        if self.PCA:
          image_testarr=cv2.resize(image_testarr,(150,150))
        if not self.PCA:
          image_testarr=cv2.resize(image_array,(128,128))
        self.testData.append([image_testarr,category])

    random.shuffle(self.trainData)
    random.shuffle(self.testData)
  
  def PrincipalCompAnalysis(self):
    self.pcXtrain,self.pcytrain=[],[]
    self.pcXtest,self.pcytest=[],[]

    for features,category in self.trainData:
      self.pcXtrain.append(np.ravel(features))
      self.pcytrain.append(category)

    for features,category in self.testData:
      self.pcXtest.append(np.ravel(features))
      self.pcytest.append(np.ravel(category))

    self.pcXtrain,self.pcXtest=np.array(self.pcXtrain),np.array(self.pcXtest)

    self.pcXtrain=self.pcXtrain/255.0
    self.pcXtest=self.pcXtest/255.0
    pca=PCA(n_components=0.95)
    self.pcaTra=pca.fit_transform(self.pcXtrain)
    self.pcaTes=pca.transform(self.pcXtest)
    
  
  def featLabel(self): #splits data into features and labels (X,y)
    self.X_train,self.y_train=[],[]
    self.X_test,self.y_test=[],[]

    for features,category in self.trainData:
      self.X_train.append(features)
      self.y_train.append(category)

    for features,category in self.testData:
      self.X_test.append(features)
      self.y_test.append(category)
      
    self.X_train,self.X_test=np.array(self.X_train).reshape(-1,128,128,1),np.array(self.X_test).reshape(-1,128,128,1)
    self.y_train,self.y_test=np.array(self.y_train),np.array(self.y_test)
  
  def CNN(self):

    if not self.PCA:
      self.X_train,self.X_test=self.X_train.astype('float32'),self.X_test.astype('float32')

      self.y_train,self.y_test=keras.utils.to_categorical(self.y_train,10),keras.utils.to_categorical(self.y_test,10)
      self.X_train,self.X_test=self.X_train/255.0,self.X_test/255.0

      model1 = Sequential()

      model1.add(Conv2D(128,(3,3),padding='same',activation='relu',input_shape=self.X_train.shape[1:]
                     ,kernel_regularizer=regularizers.L1(0.01)))
      model1.add(Conv2D(128,(3,3)))
      model1.add(MaxPooling2D((2,2)))
      model1.add(Dropout(0.25))

      model1.add(Conv2D(256,(3,3),padding='same',activation='relu',
                     kernel_regularizer=regularizers.L1(0.01)))
      model1.add(Conv2D(256,(3,3),activation='relu'))
      model1.add(MaxPooling2D((2,2)))
      model1.add(Dropout(0.25))

      model1.add(Flatten())

      model1.add(Dense(512,activation='relu',kernel_regularizer=regularizers.L1(0.01)))
      model1.add(Dropout(0.5))
      model1.add(Dense(10,activation='softmax'))

      opt = keras.optimizers.Adam(learning_rate=0.00001)
      model1.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

      self.history=model1.fit(self.X_train,self.y_train,batch_size=32,epochs=100,
              validation_data=(self.X_test,self.y_test),shuffle=True)

      model1.summary()
      self.plot()

    elif self.PCA:
      model2=Sequential()

      model2.add(Conv1D(128, 2, activation='relu', input_shape=(self.pcaTra.shape[1],)))
      model2.add(MaxPooling1D(2))

      model2.add(Dense(64, activation="relu"))

      model2.add(Flatten())

      model2.add(Dense(10, activation = 'softmax'))

      opt=keras.optimizers.Adam(learning_rate=0.0001)
      model2.compile(loss = 'categorical_crossentropy', 
      optimizer = opt,metrics = ['accuracy'])

      self.history=model2.fit(self.pcaTra,self.pcytrain,batch_size=32,epochs=100,
              validation_data=(self.pcaTes,self.pcytest),shuffle=True)

      model2.summary()
      self.plot()
    
    
  def plot(self):
    plt.plot(self.history.history['accuracy'])
    plt.plot(self.history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epochs')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

    print("\n")

    plt.plot(self.history.history['loss'])
    plt.plot(self.history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

  def execute(self):
    self.preprocessData()
    self.PrincipalCompAnalysis()
    if not self.PCA:
      self.featLabel()
    self.CNN()

In [ ]:
conv1=ConvoNeuralNets()

In [ ]:
conv2=ConvoNeuralNets(PCA=True)